## Imports

In [1]:
import torch
from tqdm import tqdm
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline 
# %matplotlib inline to creating seperate window for matplotlib figures it will draw it in cell output instead

## Read Dataset

In [2]:
words = open("/kaggle/input/eureka-labs/names.txt","r").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

## char to id

In [4]:
chars     = sorted(list(set(''.join(words))))
stoi      = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos      = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


## Dataset

In [5]:
def build_dataset(words):
    block_size = 3 # context length : how many characters do we take to predict next one
    X, Y       = [],[]

    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '----->', itos[ix])
            context = context[1:] + [ix] #crop and apppend

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    
    return X,Y

import random
random.seed(42)
random.shuffle(words)

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr   = build_dataset(words[:n1])
Xval, Yval = build_dataset(words[n1:n2])
Xte, Yte   = build_dataset(words[n2])

## Initialize weights

In [6]:
n_hidden   = 64 # the number of neurons in the hidden layer of the MLP
vocab_size = 27
n_embd     = 10 # the dimensionality of the character embedding vectors
block_size = 3

g = torch.Generator().manual_seed(2147483647) # for reproducability
C = torch.randn((vocab_size, n_embd), generator=g)

W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size) ** 0.5) #kamming dist 
b1 = torch.randn((n_hidden), generator=g) * 0.1 #for making initial weights near to zero to mask un uinform distribution

W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn((vocab_size), generator=g) * 0.1

bngain = torch.ones((1,n_hidden)) * 0.1 + 0.1   #Batch normalization trainable parameter
bnbias = torch.zeros((1,n_hidden)) * 0.1  #Batch normalization trainable parameter

bnmean_running = torch.zeros((1,n_hidden))
bnstd_running  = torch.ones((1,n_hidden))

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

## Training loop

In [7]:
batch_size = 32
n = batch_size
# minibatch
ix     = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [8]:
#forward pass "chunkated" into smaller steps that are possible to backward one at a time
emb     = C[Xb] #embed the characters into the vectors
embcat  = emb.view(emb.shape[0], -1) # concate the vectors

#Linear layer 1
hprebn  = embcat @ W1 + b1 #hidden layer pre activation

#------------------------------- BatchNorm layer -------------------------------
bnmeani   = 1/n*hprebn.sum(0, keepdim=True)
bndiff    = hprebn - bnmeani
bndiff2   = bndiff**2
bnvar     = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5 #?? (bnvar + 0.00001)/2
bnraw     = bndiff * bnvar_inv
hpreact   = bngain * bnraw + bnbias
#------------------------------------------------------------------------------
#Non linearity
h       = torch.tanh(hpreact) #hidden layer

#Linear layer 2
logits  = h @ W2 + b2 # output layer
#print(logits[0])

#-----------------------------loss function------------------------------------------------

logit_maxes    = logits.max(1, keepdim=True).values
norm_logits    = logits - logit_maxes # subtract max for numerical stability cause high values lead to instability as we are exponentiating them
#print(norm_logits[0])
counts         = norm_logits.exp() #taking natural(e^x) exponent of all values from tensor
#print(counts)
counts_sum     = counts.sum(1, keepdims=True)
# print(counts_sum)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# print(counts_sum_inv)
probs          = counts * counts_sum_inv
# print(probs)
logprobs       = probs.log() #natural logaritham of all elements from the tensor.
#print(logprobs) #logits become -ve now
loss           = -logprobs[range(n), Yb].mean()
#-----------------------------------------------------------------------------

#Backward pass
for p in parameters:
    p.grad = None
    
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.2848, grad_fn=<NegBackward0>)

In [9]:
# logprobs[(0,1),(0,1)]

In [10]:
#utility function we are going to use later for comparing manual gradients with PyTorch gradients
def cmp(s, dt, t):
    ex      = torch.all(dt == t.grad).item() # Extractly equal : checking if all values are equal by precision also if yes it retruns boolean value as .item()
    app     = torch.allclose(dt, t.grad)     # Approximatly equal by precision level is ignored
    maxdiff = (dt - t.grad).abs().max().item() # highest absolute diffrence
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [11]:
#norm_logits    = logits - logit_maxes
# c11 c12 c13 = a11 a12 a13   b1
# c21 c22 c23 = a21 a22 a23 - b2
# c31 c32 c33 = a31 a32 a33   b3

#so e.g C32 = a32 - b3

#norm_logits    = logits - logit_maxes - this operation is for numerical stability only so it dose not impact the 
# loss at all so it's derivative is zero

In [12]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs               = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n                          #d(loss)/d(logprobs) = -1/n
dprobs                  = (1.0 / probs) * dlogprobs         #d/dx log(x)         = 1/x   # Boosting the gradients where probs is low if it is 1 then we just pass grads in back layer
dcounts_sum_inv         = (counts * dprobs).sum(1, keepdim=True) # count : (32,27) * (32,1) using broadcasting for sum operation
dcounts                 = counts_sum_inv * dprobs
dcounts_sum             = (-counts_sum**-2) * dcounts_sum_inv #d(x**-1)/dx = -1/x**2 
dcounts                += torch.ones_like(counts) * dcounts_sum
dnorm_logits            = counts * dcounts                    #d(e^x)/dx = x     
dlogits                 = dnorm_logits.clone()
dlogit_maxes            = (-dnorm_logits).sum(1, keepdim=True) #drivaties flow as it is through nodes for -ve and +ve operations.
dlogits                += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

dh                      = dlogits @ W2.T # for more clarification see the video : 51:00 hrs
dW2                     = h.T @ dlogits  # --
db2                     = dlogits.sum(0) # --

dhpreact                = (1.0 - h**2) * dh # d(tanh(x))/dx = 1 - tanh(x) ** 2

dbngain                 = (bnraw * dhpreact).sum(0, keepdim=True)
dbnraw                  = bngain * dhpreact
dbnbias                 = dhpreact.sum(0, keepdim=True)

cmp('logprobs',       dlogprobs,        logprobs)
cmp('probs',          dprobs,           probs)
cmp('counts_sum_inv', dcounts_sum_inv,  counts_sum_inv)
cmp('counts_sum',     dcounts_sum,      counts_sum)
cmp('counts',         dcounts,          counts)
cmp('logit_maxes',    dlogit_maxes,     logit_maxes)
cmp('logits',         dlogits,          logits)

cmp('h',              dh,               h)
cmp('W2',             dW2,              W2)
cmp('b2',             db2,              b2)

cmp('hpreact',        dhpreact,         hpreact)

cmp('bngain',         dbngain,          bngain)
cmp('bnraw',          dbnraw,           bnraw)
cmp('bnbias',         dbnbias,          bnbias)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
bngain          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw           | exact: False | approximate: True  | maxdiff: 1.1641532182693481e-10
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298